In [3]:
!pip install torchtext

     |████████████████████████████████| 64 kB 2.1 MB/s 
     |████████████████████████████████| 1.1 MB 7.1 MB/s 


In [2]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
import torch 
import torch.nn as nn 
import torch.optim as optim
import torch.autograd as autograd
import torch.nn.functional as F 
import torchtext 
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [2]:
col_names = ['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP']
df_news = pd.read_csv('/Users/ryomisawa/Downloads/NewsAggregatorDataset/newsCorpora.csv', sep = '\t', names=col_names)

df_news = df_news[(df_news['PUBLISHER'] == 'Reuters') | (df_news['PUBLISHER'] ==  'Huffington Post')|(df_news['PUBLISHER'] == 'Businessweek') | (df_news['PUBLISHER'] == 'Contactmusic.com') |(df_news['PUBLISHER'] == 'Daily Mail')].sample(frac=1, random_state=0).reset_index()

In [ ]:
df_news.drop('ID', axis=1, inplace=True)
df_news.drop('URL', axis=1, inplace=True)
df_news.drop('PUBLISHER', axis=1, inplace=True)
df_news.drop('STORY', axis=1, inplace=True)
df_news.drop('HOSTNAME', axis=1, inplace=True)
df_news.drop('TIMESTAMP', axis=1, inplace=True)
df_news.drop('index', axis=1, inplace=True)

from sklearn.model_selection import train_test_split

df_news_train, df_news_test = train_test_split(df_news, train_size=0.9)

import os 
os.chdir('/Users/ryomisawa/nlp_tutorial')
df_news_train.to_csv('news_train.csv')
df_news_test.to_csv('news_test.csv')

In [3]:
df_x = df_news['TITLE']
df_y = df_news['CATEGORY']

In [4]:
df_train, df_valid, y_train, y_valid = train_test_split(df_x, df_y, test_size=0.2)
df_valid, df_test, y_valid, y_test = train_test_split(df_valid, y_valid, test_size=0.5)

In [ ]:
df_tra

In [5]:
#前処理
import string
import re

def preprocessing_text(text):
  text = re.sub('<br />', '', text)

  for p in string.punctuation:
      text = text.replace(p, ' ')
  return text

def tokenizer_punctuation(text):
  return text.strip().split()

def tokenizer_with_preprocessing(text):
  text = preprocessing_text(text)
  ret = tokenizer_punctuation(text)
  return ret 

In [6]:
import collections
texts = tokenizer_with_preprocessing(df_train.to_string(index=False))
texts = collections.Counter(texts).most_common()

text_id = {}
for i in range(len(texts)):
    if texts[i][1] > 1:
        text_id[texts[i][0]] = i+1
    else:
        text_id[texts[i][0]] = 0

In [31]:
df_train, df_nouse, y_train, y_nouse = train_test_split(df_x, df_y, test_size=0.5)

In [32]:
def word2id(text):
    ids = []
    for word in text:
        if word in text_id:
            ids.append(text_id[word])
        else:
            ids.append(0)
    return ids

In [163]:
x_train_id = []
for sentence in df_train:
    sentence = tokenizer_with_preprocessing(sentence)
    x_train_id.append(word2id(sentence))
x_valid_id = []
for sentence in df_valid:
    sentence = tokenizer_with_preprocessing(sentence)
    x_valid_id.append(word2id(sentence))
#x_valid_oh = torch.tensor(x_valid_oh)
x_test_id = []
for sentence in df_test:
    sentence = tokenizer_with_preprocessing(sentence)
    x_test_id.append(word2id(sentence))

In [164]:
max_length = max([len(s) for s in x_train_id])
for i in range(len(x_train_id)):
    x_train_id[i] = x_train_id[i] + [0 for i in range(max_length - len(x_train_id[i]))]
x_train_id = torch.tensor(x_train_id)

In [285]:
max_length = max([len(s) for s in x_valid_id])
for i in range(len(x_valid_id)):
    x_valid_id[i] = x_valid_id[i] + [0 for i in range(max_length - len(x_valid_id[i]))]
x_valid_id = torch.tensor(x_valid_id)

In [286]:
max_length = max([len(s) for s in x_test_id])
for i in range(len(x_test_id)):
    x_test_id[i] = x_test_id[i] + [0 for i in range(max_length - len(x_test_id[i]))]
x_test_id = torch.tensor(x_test_id)

In [287]:
len_seq_train = torch.tensor([len(s) for s in x_train_id]) 
len_seq_valid = torch.tensor([len(s) for s in x_valid_id])
len_seq_test = torch.tensor([len(s) for s in x_test_id])

In [14]:
max_length = max(text_id.values())
x_train_oh = []
for sentence in df_train:
    sentence = tokenizer_with_preprocessing(sentence)
    x_train_oh.append(np.identity(max_length+1)[word2id(sentence)])
x_valid_oh = []
for sentence in df_valid:
    sentence = word2id(tokenizer_with_preprocessing(sentence))
    x_valid_oh.append(np.identity(max_length+1)[word2id(sentence)])
#x_valid_oh = torch.tensor(x_valid_oh)
x_test_oh = []
for sentence in df_test:
    sentence = word2id(tokenizer_with_preprocessing(sentence))
    x_test_oh.append(np.identity(max_length+1)[word2id(sentence)])
#x_test_oh = torch.tensor(x_test_oh)

KeyboardInterrupt: 

In [220]:
t_train = []
t_valid = []
t_test = []
category = ['b', 'e', 't', 'm']
number = [0, 1, 2, 3]
n_labels = len(number)
for text in y_train.replace(category, number):
    t_train.append(text)
t_train = torch.from_numpy(np.eye(n_labels)[t_train]).long()
for text in y_valid.replace(category, number):
    t_valid.append(text)
t_valid = torch.from_numpy(np.eye(n_labels)[t_valid]).long()
for text in y_test.replace(category, number):
    t_test.append(text)
t_test = torch.from_numpy(np.eye(n_labels)[t_test]).long()

In [107]:
rng = np.random.RandomState(1234)
random_state = 42

In [17]:
class Embedding(nn.Module):
    def __init__(self, emb_dim, vocab_size):
        super().__init__()
        self.embedding_matrix = nn.Parameter(torch.rand((vocab_size, emb_dim), dtype=torch.float))
    
    def forward(self, x):
        return F.embedding(x, self.embedding_matrix)

In [393]:
embedding_matrix = nn.Parameter(torch.rand((word_num, emb_dim), dtype=torch.fl))
h = F.embedding(x, embedding_matrix)

In [399]:
len(h[0])

19

In [400]:
len(x[0])

19

In [479]:
emb_dim = 300
hid_dim = 50
glorot = 6/(emb_dim + hid_dim*2)
W = nn.Parameter(torch.tensor(rng.uniform(
                       low=-np.sqrt(glorot),
                       high=np.sqrt(glorot),
                       size=(emb_dim + hid_dim, hid_dim)
                    ).astype('float32')))
b = nn.Parameter(torch.tensor(np.zeros([hid_dim]).astype('float32')))

In [488]:
class RNN(nn.Module):
    def __init__(self, in_dim, hid_dim):
        super().__init__()
        self.hid_dim = hid_dim
        glorot = 6/(in_dim + hid_dim*2)
        self.W = nn.Parameter(torch.tensor(rng.uniform(
                        low=-np.sqrt(glorot),
                        high=np.sqrt(glorot),
                        size=(in_dim + hid_dim, hid_dim)
                    ).astype('float32')))
        self.b = nn.Parameter(torch.tensor(np.zeros([hid_dim]).astype('float32')))

    def function(self, h, x):
        return torch.tanh(torch.matmul(torch.cat([h, x], dim=1), self.W) + self.b)

    def forward(self, x, len_seq_max=0, init_state=None):
        x = x.transpose(0, 1)  # 系列のバッチ処理のため、次元の順番を「系列、バッチ」の順に入れ替える
        state = init_state
        
        if init_state is None:  # 初期値を設定しない場合は0で初期化する
            state = torch.zeros((x[0].size()[0], self.hid_dim)).to(x.device)

        size = list(state.unsqueeze(0).size())
        size[0] = 0
        output = torch.empty(size, dtype=torch.float).to(x.device)  # 一旦空テンソルを定義して順次出力を追加する

        if len_seq_max == 0:
            len_seq_max = x.size(0)
        
        for i in range(len_seq_max):
            state = torch.tanh(torch.matmul(torch.cat([state, x[i]], dim=1), W) + b)
            output = torch.cat([output, state.unsqueeze(0)])
        return output

In [ ]:
state = torch.zeros((x[0].size()[0], hid_dim)).to(x.device)
for i in range(len_seq_max):
    state = torch.tanh(torch.matmul(torch.cat([state, x[i]], dim=1), W) + b)
    output = torch.cat([output, state.unsqueeze(0)])  # 出力系列の追加

In [472]:
x = h.transpose(0, 1)

In [474]:
size = list(state.unsqueeze(0).size())
size[0] = 0

In [476]:
output = torch.empty(size, dtype=torch.float).to(x.device)

In [483]:
state = torch.zeros((x[0].size()[0], hid_dim)).to(x.device)
for i in range(len_seq_max):
    state = torch.tanh(torch.matmul(torch.cat([state, x[i]], dim=1), W) + b)
    output = torch.cat([output, state.unsqueeze(0)])  # 出力系列の追加

In [464]:
for i in range(len_seq_max):
    print(x[i])

tensor([[0.1439, 0.1174, 0.8272,  ..., 0.8164, 0.2739, 0.2553],
        [0.5425, 0.3298, 0.5337,  ..., 0.0745, 0.5213, 0.2803],
        [0.4411, 0.0068, 0.5785,  ..., 0.4814, 0.6679, 0.9264],
        ...,
        [0.4956, 0.2459, 0.0520,  ..., 0.5274, 0.1826, 0.2962],
        [0.5804, 0.5421, 0.3756,  ..., 0.1501, 0.0254, 0.0936],
        [0.3551, 0.1195, 0.9951,  ..., 0.7573, 0.7799, 0.0235]],
       grad_fn=<SelectBackward>)
tensor([[0.7357, 0.7829, 0.4624,  ..., 0.5037, 0.3396, 0.8498],
        [0.2736, 0.6062, 0.2357,  ..., 0.5083, 0.6638, 0.4232],
        [0.1990, 0.8759, 0.7293,  ..., 0.5510, 0.5565, 0.1864],
        ...,
        [0.4211, 0.5575, 0.1450,  ..., 0.2999, 0.3643, 0.6648],
        [0.1146, 0.7505, 0.1310,  ..., 0.6835, 0.8934, 0.7929],
        [0.3097, 0.5993, 0.7906,  ..., 0.0590, 0.2146, 0.6339]],
       grad_fn=<SelectBackward>)
tensor([[0.3080, 0.1528, 0.3591,  ..., 0.4341, 0.2842, 0.6763],
        [0.4535, 0.9851, 0.9125,  ..., 0.2180, 0.9269, 0.2111],
        [0

In [177]:
len_seq_max = torch.max(len_seq_train)

In [182]:
state = torch.zeros((hid_dim, x[0].size()[0],)).to(device)

In [181]:
outputs = []
for s in x:
    for i in range(len_seq_max):
        state = torch.tanh(torch.matmul(torch.cat([state, s[i]], dim=1), W) + b)
        output = torch.cat([output, state.unsqueeze(0)])
    outputs.append(output)

RuntimeError: zero-dimensional tensor (at position 1) cannot be concatenated

In [390]:
class RNN(nn.Module):
    def __init__(self, in_dim, hid_dim):
        super().__init__()
        self.hid_dim = hid_dim
        glorot = 6/(in_dim + hid_dim*2)
        self.W = nn.Parameter(torch.tensor(rng.uniform(
                        low=-np.sqrt(glorot),
                        high=np.sqrt(glorot),
                        size=(in_dim + hid_dim, hid_dim)
                    ).astype('float32')))
        self.b = nn.Parameter(torch.tensor(np.zeros([hid_dim]).astype('float32')))

    def function(self, h, x):
        return torch.tanh(torch.matmul(torch.cat([h, x], dim=1), self.W) + self.b)

    def forward(self, x, len_seq_max=0, init_state=None):
        x = x.transpose(0, 1)  # 系列のバッチ処理のため、次元の順番を「系列、バッチ」の順に入れ替える
        state = init_state
        
        if init_state is None:  # 初期値を設定しない場合は0で初期化する
            state = torch.zeros((x[0].size()[0], self.hid_dim)).to(x.device)

        size = list(state.unsqueeze(0).size())
        size[0] = 0
        output = torch.empty(size, dtype=torch.float).to(x.device)  # 一旦空テンソルを定義して順次出力を追加する

        if len_seq_max == 0:
            len_seq_max = x.size(0)
        for i in range(len_seq_max):
            state = self.function(state, x[i])
            output = torch.cat([output, state.unsqueeze(0)])  # 出力系列の追加
        return output
class SequenceTaggingNet(nn.Module):
    def __init__(self, word_num, emb_dim, hid_dim):
        super().__init__()
        self.emb = Embedding(emb_dim, word_num)
        self.rnn = RNN(emb_dim, hid_dim)
        self.linear = nn.Linear(hid_dim, 4)
    
    def forward(self, x, len_seq_max=0, len_seq=None, init_state=None):
        h = self.emb(x)
        h = self.rnn(h, len_seq_max, init_state)
        if len_seq is not None:
            h = h[len_seq-1, list(range(len(x))), :]
        else:
            h = h[-1]
        y = self.linear(h)
        return y

In [288]:
x_train_id = x_train_id.long()

In [289]:
x_valid_id = torch.tensor(x_valid_id).long()
x_test_id = torch.tensor(x_test_id).long()

In [81]:
def torch_log(x):
    return torch.log(torch.clamp(x, min=1e-10))

In [ ]:
from sklearn.metrics import accuracy_score

In [304]:
word_num = len(texts)
emb_dim = 300
hid_dim = 50
n_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = SequenceTaggingNet(word_num, emb_dim, hid_dim)
net.to(device)
optimizer = optim.Adam(net.parameters(), lr=0.001)
softmax = nn.Softmax(dim=1)
for epoch in range(n_epochs):
    net.train()
    #n_train = 0
    #acc_train = 0
    net.zero_grad()

    t = t_train.to(device)
    x = x_train_id.to(device)
    h = net(x, torch.max(len_seq_train), len_seq_train)
    y = softmax(h).squeeze()

    loss_train = -torch.mean(t*torch_log(y) + (1-t)*torch_log(1-y))

    loss_train.backward()  # 誤差の逆伝播
        
    optimizer.step()  # パラメータの更新

    acc_train = accuracy_score(t.argmax(axis=1), y.argmax(axis=1))

    net.eval()

    t = t_test.to(device)
    x = x_test_id.to(device)
    h = net(x, torch.max(len_seq_test), len_seq_test)
    y = softmax(h).squeeze()

    loss_test = -torch.mean(t*torch_log(y) + (1-t)*torch_log(1-y))
    
    acc_test = accuracy_score(t.argmax(axis=1), y.argmax(axis=1))
    print('loss_train:{:.3f}, accuracy_train:{:.3f},loss_test:{:.3f}, accuracy_test:{:.3f} '.format(loss_train, acc_train, loss_test, acc_test))

loss_train:0.532, accuracy_train:0.425,loss_test:0.504, accuracy_test:0.395 


KeyboardInterrupt: 

In [490]:
import time
word_num = len(texts)
emb_dim = 300
hid_dim = 50
n_epochs = 10

num_data_train = len(x_train_id)
num_data_test = len(x_test_id)
batch_size = 8

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = SequenceTaggingNet(word_num, emb_dim, hid_dim)
net.to(device)
optimizer = optim.Adam(net.parameters(), lr=0.01)
softmax = nn.Softmax(dim=1)
for epoch in range(n_epochs):
    
    t1 = time.time()
    sff_idx_train = np.random.permutation(num_data_train)
    sff_idx_test = np.random.permutation(num_data_test)

    loss_train = []
    loss_test = []
    acc_train = []
    acc_test = []

    net.train()
    for idx in range(0, num_data_train, batch_size):
        net.zero_grad()

        x = x_train_id[sff_idx_train[idx:idx + batch_size if idx + batch_size < num_data_train else num_data_train]].to(device)
        t = t_train[sff_idx_train[idx:idx + batch_size if idx + batch_size < num_data_train else num_data_train]].to(device)
        len_seq_train = torch.tensor([len(s) for s in x])
        h = net(x, torch.max(len_seq_train), len_seq_train)
        y = softmax(h).squeeze()

        l_train = -torch.mean(t*torch_log(y) + (1-t)*torch_log(1-y))
        l_train.backward()
        optimizer.step()

        loss_train.append(l_train.item())
        acc_train.append(accuracy_score(t.argmax(axis=1), y.argmax(axis=1)))
    
    net.eval()
    for idx in range(0, num_data_test, batch_size):
        x = x_test_id[sff_idx_test[idx:idx + batch_size if idx + batch_size < num_data_test else num_data_test]].to(device)
        t = t_test[sff_idx_test[idx:idx + batch_size if idx + batch_size < num_data_test else num_data_test]].to(device)
        len_seq_test = torch.tensor([len(s) for s in x])
        h = net(x, torch.max(len_seq_test), len_seq_test)
        y = softmax(h).squeeze()
        l_test = -torch.mean(t*torch_log(y) + (1-t)*torch_log(1-y))

        loss_test.append(l_test.item())
        acc_test.append(accuracy_score(t.argmax(axis=1), y.argmax(axis=1)))
    print('loss_train:{:.3f}, accuracy_train:{:.3f},loss_test:{:.3f}, accuracy_test:{:.3f} '.format(np.mean(loss_train), np.mean(acc_train), np.mean(loss_test), np.mean(acc_test)))

KeyboardInterrupt: 

In [320]:
import gensim
model = gensim.models.KeyedVectors.load_word2vec_format('/Users/ryomisawa/Downloads/GoogleNews-vectors-negative300.bin', binary=True)

In [322]:
df_train

1441     NML Slams Argentina's 'Brazen Step' to Pay Bon...
11564    17 Times Zach Braff's 'Wish I Was Here' Refere...
7339     Japan's Exports Decline in May on Weak US, Asi...
10795    REFILE-China July official PMI rises to 51.7 f...
3185     UPDATE 2-Bank of America ordered to pay $1.27 ...
                               ...                        
9253     UPDATE 2-US top court mostly upholds Obama bid...
13265    Spotify's Most Popular Song Is Also The Most T...
11983    Tila Tequila, Former Reality Star & Glamour Mo...
8207     Jobless Claims in US Drop to Lowest Level Sinc...
11277    Canceled And Renewed TV Shows For The 2014-'15...
Name: TITLE, Length: 6670, dtype: object

In [403]:
len(h[1])

19

In [405]:
len(h[6])

19

In [451]:
d = 300

train_embedding = []
max_length_train = max([len(s.split()) for s in df_train])

In [452]:
for s in df_train:
    s = s.split()
    x = []
    for w in s:
        if w in model:
            x.append(model[w])
        else:
            x.append(np.zeros(d))
    for i in range(max_length - len(x)):
        x.append(np.zeros(d))
    train_embedding.append(x)
train_embedding = torch.tensor(train_embedding)

In [458]:
train_embedding =train_embedding.float()

In [453]:
d = 300
valid_embedding = []
max_length_valid = max([len(s.split()) for s in df_valid])

In [454]:
for s in df_valid:
    s = s.split()
    x = []
    for w in s:
        if w in model:
            x.append(model[w])
        else:
            x.append(np.zeros(d))
    for i in range(max_length_valid - len(x)):
        x.append(np.zeros(d))
    valid_embedding.append(x)
valid_embedding = torch.tensor(valid_embedding)

In [515]:
test_embedding =test_embedding.float()

In [455]:
d = 300
test_embedding = []
max_length_test = max([len(s.split()) for s in df_test])

In [456]:
for s in df_test:
    s = s.split()
    x = []
    for w in s:
        if w in model:
            x.append(model[w])
        else:
            x.append(np.zeros(d))
    for i in range(max_length_test - len(x)):
        x.append(np.zeros(d))
    test_embedding.append(x)
test_embedding = torch.tensor(test_embedding)

In [371]:
"""
class RNN(nn.Module):
    def __init__(self, in_dim, hid_dim):
        super().__init__()
        self.hid_dim = hid_dim
        glorot = 6/(in_dim + hid_dim*2)
        self.W = nn.Parameter(torch.tensor(rng.uniform(
                        low=-np.sqrt(glorot),
                        high=np.sqrt(glorot),
                        size=(in_dim + hid_dim, hid_dim)
                    ).astype('float32')))
        self.b = nn.Parameter(torch.tensor(np.zeros([hid_dim]).astype('float32')))

    def function(self, h, x):
        return torch.tanh(torch.matmul(torch.cat([h, x], dim=1), self.W) + self.b)

    def forward(self, x, len_seq_max=0, init_state=None):
        x = x.transpose(0, 1)  # 系列のバッチ処理のため、次元の順番を「系列、バッチ」の順に入れ替える
        state = init_state
        
        if init_state is None:  # 初期値を設定しない場合は0で初期化する
            state = torch.zeros((x[0].size()[0], self.hid_dim)).to(x.device)

        size = list(state.unsqueeze(0).size())
        size[0] = 0
        output = torch.empty(size, dtype=torch.float).to(x.device)  # 一旦空テンソルを定義して順次出力を追加する

        if len_seq_max == 0:
            len_seq_max = x.size(0)
        for s in x:
            state = self.function(state, s)
            output = torch.cat([output, state.unsqueeze(0)])  # 出力系列の追加
        return output

In [486]:
class SequenceTaggingNet2(nn.Module):
    def __init__(self, word_num, emb_dim, hid_dim):
        super().__init__()
        self.rnn = RNN(emb_dim, hid_dim)
        self.linear = nn.Linear(hid_dim, 4)
    
    def forward(self, x, embedding, len_seq_max=0, len_seq=None, init_state=None):
        h = embedding
        h = self.rnn(h, len_seq_max, init_state)
        if len_seq is not None:
            h = h[len_seq-1, list(range(len(x))), :]
        else:
            h = h[-1]
        y = self.linear(h)
        return y

In [516]:
word_num = len(texts)
emb_dim = 300
hid_dim = 50
n_epochs = 10

num_data_train = len(x_train_id)
num_data_test = len(x_test_id)
batch_size = 8

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = SequenceTaggingNet2(word_num, emb_dim, hid_dim)
net.to(device)
optimizer = optim.Adam(net.parameters(), lr=0.01)
softmax = nn.Softmax(dim=1)

for epoch in range(n_epochs):
    
    t1 = time.time()
    sff_idx_train = np.random.permutation(num_data_train)
    sff_idx_test = np.random.permutation(num_data_test)

    loss_train = []
    loss_test = []
    acc_train = []
    acc_test = []

    net.train()
    for idx in range(0, num_data_train, batch_size):
        net.zero_grad()

        x = x_train_id[sff_idx_train[idx:idx + batch_size if idx + batch_size < num_data_train else num_data_train]].to(device)
        embedding = train_embedding[sff_idx_train[idx:idx + batch_size if idx + batch_size < num_data_train else num_data_train]].to(device)
        t = t_train[sff_idx_train[idx:idx + batch_size if idx + batch_size < num_data_train else num_data_train]].to(device)
        len_seq_train = torch.LongTensor([len(s) for s in x])

        h = net(x, embedding, torch.max(len_seq_train), len_seq_train)
        y = softmax(h).squeeze()

        l_train = -torch.mean(t*torch_log(y) + (1-t)*torch_log(1-y))
        l_train.backward()
        optimizer.step()

        loss_train.append(l_train.item())
        acc_train.append(accuracy_score(t.argmax(axis=1), y.argmax(axis=1)))
        
        net.eval()
    for idx in range(0, num_data_test, batch_size):
        x = x_test_id[sff_idx_test[idx:idx + batch_size if idx + batch_size < num_data_test else num_data_test]].to(device)
        embedding = test_embedding[sff_idx_test[idx:idx + batch_size if idx + batch_size < num_data_test else num_data_test]].to(device)
        t = t_test[sff_idx_test[idx:idx + batch_size if idx + batch_size < num_data_test else num_data_test]].to(device)
        len_seq_test = torch.LongTensor([len(s) for s in x])

        h = net(x, embedding, torch.max(len_seq_test), len_seq_test)
        y = softmax(h).squeeze()

        l_test = -torch.mean(t*torch_log(y) + (1-t)*torch_log(1-y))

        loss_test.append(l_test.item())
        acc_test.append(accuracy_score(t.argmax(axis=1), y.argmax(axis=1)))
    print('loss_train:{:.3f}, accuracy_train:{:.3f},loss_test:{:.3f}, accuracy_test:{:.3f} '.format(np.mean(loss_train), np.mean(acc_train), np.mean(loss_test), np.mean(acc_test)))

loss_train:0.491, accuracy_train:0.434,loss_test:0.489, accuracy_test:0.463 
loss_train:0.484, accuracy_train:0.472,loss_test:0.487, accuracy_test:0.403 
loss_train:0.483, accuracy_train:0.485,loss_test:0.487, accuracy_test:0.425 
loss_train:0.482, accuracy_train:0.488,loss_test:0.486, accuracy_test:0.507 
loss_train:0.481, accuracy_train:0.498,loss_test:0.485, accuracy_test:0.503 
loss_train:0.480, accuracy_train:0.497,loss_test:0.486, accuracy_test:0.503 
loss_train:0.480, accuracy_train:0.501,loss_test:0.486, accuracy_test:0.500 
loss_train:0.479, accuracy_train:0.503,loss_test:0.487, accuracy_test:0.506 
loss_train:0.479, accuracy_train:0.507,loss_test:0.489, accuracy_test:0.470 
loss_train:0.479, accuracy_train:0.508,loss_test:0.488, accuracy_test:0.501 


In [517]:
class RNN(nn.Module):
    def __init__(self, in_dim, hid_dim):
        super().__init__()
        self.hid_dim = hid_dim
        glorot = 6/(in_dim + hid_dim*2)
        self.W = nn.Parameter(torch.tensor(rng.uniform(
                        low=-np.sqrt(glorot),
                        high=np.sqrt(glorot),
                        size=(in_dim + hid_dim, hid_dim)
                    ).astype('float32')))
        self.b = nn.Parameter(torch.tensor(np.zeros([hid_dim]).astype('float32')))

    def function(self, h, x):
        return torch.tanh(torch.matmul(torch.cat([h, x], dim=1), self.W) + self.b)

    def forward(self, x, len_seq_max=0, init_state=None):
        x = x.transpose(0, 1)  # 系列のバッチ処理のため、次元の順番を「系列、バッチ」の順に入れ替える
        state = init_state
        
        if init_state is None:  # 初期値を設定しない場合は0で初期化する
            state = torch.zeros((x[0].size()[0], self.hid_dim)).to(x.device)

        size = list(state.unsqueeze(0).size())
        size[0] = 0
        output = torch.empty(size, dtype=torch.float).to(x.device)  # 一旦空テンソルを定義して順次出力を追加する

        if len_seq_max == 0:
            len_seq_max = x.size(0)
        
        for i in range(len_seq_max):
            state = torch.tanh(torch.matmul(torch.cat([state, x[i]], dim=1), W) + b)
            output = torch.cat([output, state.unsqueeze(0)])
        return output

In [521]:
class SequenceTaggingNet2(nn.Module):
    def __init__(self, word_num, emb_dim, hid_dim):
        super().__init__()
        self.rnn_1 = RNN(emb_dim, hid_dim, bidirectional=True)
        self.rnn_2 = RNN(hid_dim, hid_dim, bidirectional=True)
        self.linear = nn.Linear(hid_dim, 4)
    
    def forward(self, x, embedding, len_seq_max=0, len_seq=None, init_state=None):
        h = embedding
        h = self.rnn_1(h, len_seq_max, init_state)
        if len_seq is not None:
            h = h[len_seq-1, list(range(len(x))), :]
        else:
            h = h[-1]
        y = self.linear(h)
        return y

In [522]:
word_num = len(texts)
emb_dim = 300
hid_dim = 50
n_epochs = 10

num_data_train = len(x_train_id)
num_data_test = len(x_test_id)
batch_size = 8

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = SequenceTaggingNet2(word_num, emb_dim, hid_dim)
net.to(device)
optimizer = optim.Adam(net.parameters(), lr=0.01)
softmax = nn.Softmax(dim=1)

for epoch in range(n_epochs):
    
    t1 = time.time()
    sff_idx_train = np.random.permutation(num_data_train)
    sff_idx_test = np.random.permutation(num_data_test)

    loss_train = []
    loss_test = []
    acc_train = []
    acc_test = []

    net.train()
    for idx in range(0, num_data_train, batch_size):
        net.zero_grad()

        x = x_train_id[sff_idx_train[idx:idx + batch_size if idx + batch_size < num_data_train else num_data_train]].to(device)
        embedding = train_embedding[sff_idx_train[idx:idx + batch_size if idx + batch_size < num_data_train else num_data_train]].to(device)
        t = t_train[sff_idx_train[idx:idx + batch_size if idx + batch_size < num_data_train else num_data_train]].to(device)
        len_seq_train = torch.LongTensor([len(s) for s in x])

        h = net(x, embedding, torch.max(len_seq_train), len_seq_train)
        y = softmax(h).squeeze()

        l_train = -torch.mean(t*torch_log(y) + (1-t)*torch_log(1-y))
        l_train.backward()
        optimizer.step()

        loss_train.append(l_train.item())
        acc_train.append(accuracy_score(t.argmax(axis=1), y.argmax(axis=1)))
        
        net.eval()
    for idx in range(0, num_data_test, batch_size):
        x = x_test_id[sff_idx_test[idx:idx + batch_size if idx + batch_size < num_data_test else num_data_test]].to(device)
        embedding = test_embedding[sff_idx_test[idx:idx + batch_size if idx + batch_size < num_data_test else num_data_test]].to(device)
        t = t_test[sff_idx_test[idx:idx + batch_size if idx + batch_size < num_data_test else num_data_test]].to(device)
        len_seq_test = torch.LongTensor([len(s) for s in x])

        h = net(x, embedding, torch.max(len_seq_test), len_seq_test)
        y = softmax(h).squeeze()

        l_test = -torch.mean(t*torch_log(y) + (1-t)*torch_log(1-y))

        loss_test.append(l_test.item())
        acc_test.append(accuracy_score(t.argmax(axis=1), y.argmax(axis=1)))
    print('loss_train:{:.3f}, accuracy_train:{:.3f},loss_test:{:.3f}, accuracy_test:{:.3f} '.format(np.mean(loss_train), np.mean(acc_train), np.mean(loss_test), np.mean(acc_test)))

RuntimeError: size mismatch, m1: [19 x 100], m2: [350 x 50] at ../aten/src/TH/generic/THTensorMath.cpp:41

In [529]:
class SequenceTaggingNet3(nn.Module):
    def __init__(self, word_num, emb_dim, hid_dim):
        super().__init__()
        self.rnn = nn.RNN(emb_dim, hid_dim, 4, batch_first=True)
        self.linear = nn.Linear(hid_dim, 4)
    
    def forward(self,x, embedding,  len_seq_max, len_seq=None, init_state=None):
        h = embedding
        if len_seq_max > 0:
            h, _ = self.rnn(h[:, 0:len_seq_max, :], init_state)
        else:
            h, _ = self.rnn(h, init_state)
        h = h.transpose(0, 1)
        if len_seq is None:
            h = h[len_seq - 1, list(range(len(x))), :]
        else:
            h = h[-1]
        y = self.linear(h)

        return y

In [530]:
word_num = len(texts)
emb_dim = 300
hid_dim = 50
n_epochs = 10

num_data_train = len(x_train_id)
num_data_test = len(x_test_id)
batch_size = 8

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = SequenceTaggingNet3(word_num, emb_dim, hid_dim)
net.to(device)
optimizer = optim.Adam(net.parameters(), lr=0.01)
softmax = nn.Softmax(dim=1)

for epoch in range(n_epochs):
    
    t1 = time.time()
    sff_idx_train = np.random.permutation(num_data_train)
    sff_idx_test = np.random.permutation(num_data_test)

    loss_train = []
    loss_test = []
    acc_train = []
    acc_test = []

    net.train()
    for idx in range(0, num_data_train, batch_size):
        net.zero_grad()

        x = x_train_id[sff_idx_train[idx:idx + batch_size if idx + batch_size < num_data_train else num_data_train]].to(device)
        embedding = train_embedding[sff_idx_train[idx:idx + batch_size if idx + batch_size < num_data_train else num_data_train]].to(device)
        t = t_train[sff_idx_train[idx:idx + batch_size if idx + batch_size < num_data_train else num_data_train]].to(device)
        len_seq_train = torch.LongTensor([len(s) for s in x])

        h = net(x, embedding, torch.max(len_seq_train), len_seq_train)
        y = softmax(h).squeeze()

        l_train = -torch.mean(t*torch_log(y) + (1-t)*torch_log(1-y))
        l_train.backward()
        optimizer.step()

        loss_train.append(l_train.item())
        acc_train.append(accuracy_score(t.argmax(axis=1), y.argmax(axis=1)))
        
        net.eval()
    for idx in range(0, num_data_test, batch_size):
        x = x_test_id[sff_idx_test[idx:idx + batch_size if idx + batch_size < num_data_test else num_data_test]].to(device)
        embedding = test_embedding[sff_idx_test[idx:idx + batch_size if idx + batch_size < num_data_test else num_data_test]].to(device)
        t = t_test[sff_idx_test[idx:idx + batch_size if idx + batch_size < num_data_test else num_data_test]].to(device)
        len_seq_test = torch.LongTensor([len(s) for s in x])

        h = net(x, embedding, torch.max(len_seq_test), len_seq_test)
        y = softmax(h).squeeze()

        l_test = -torch.mean(t*torch_log(y) + (1-t)*torch_log(1-y))

        loss_test.append(l_test.item())
        acc_test.append(accuracy_score(t.argmax(axis=1), y.argmax(axis=1)))
    print('loss_train:{:.3f}, accuracy_train:{:.3f},loss_test:{:.3f}, accuracy_test:{:.3f} '.format(np.mean(loss_train), np.mean(acc_train), np.mean(loss_test), np.mean(acc_test)))

loss_train:0.461, accuracy_train:0.602,loss_test:0.433, accuracy_test:0.651 
loss_train:0.447, accuracy_train:0.629,loss_test:0.428, accuracy_test:0.651 
loss_train:0.447, accuracy_train:0.632,loss_test:0.427, accuracy_test:0.651 
loss_train:0.448, accuracy_train:0.639,loss_test:0.459, accuracy_test:0.652 
loss_train:0.450, accuracy_train:0.633,loss_test:0.430, accuracy_test:0.652 
loss_train:0.448, accuracy_train:0.636,loss_test:0.462, accuracy_test:0.651 
loss_train:0.451, accuracy_train:0.634,loss_test:0.441, accuracy_test:0.650 
loss_train:0.452, accuracy_train:0.622,loss_test:0.481, accuracy_test:0.651 
loss_train:0.448, accuracy_train:0.625,loss_test:0.472, accuracy_test:0.651 
loss_train:0.450, accuracy_train:0.627,loss_test:0.434, accuracy_test:0.651 


In [ ]:
rng = np.ramdom.RandomState(1234)
random_state = 42

In [ ]:
class Conv(nn.Module):
    def __init__(self, filter_shape, function=lambda x:x, stride=(1, 1), padding=1):
        super().__init__()
        fan_in = filter_shape[1] * filter_shape[2] * filter_shape[3]
        fan_out = filter_shape[0] * filter_shape[2] * filter_shape[3]
        self.W = nn.Parameter(torch.tensor(rng.uniform(
            low= -np.sqrt(6/fan_in),
            high = np.sqrt(6/fan_in),
            size=filter_shape
        ).astype('float32')))
        self.b = nn.Parameter(torch.tensor(np.zeros((filter_shape[0]), dtype='float32')))
        self.function = function
        self.stride = stride
        self.padding = padding
    def forward(self, x):
        u = F.conv2d(x, self.W, bias=self.b, stride=self.stride, padding=self.padding)
        return self.function(u)

In [658]:
d_out = 50
L = 4
b_1 = nn.Parameter(torch.tensor(np.zeros((d_out), dtype='float32')))
b_2 = nn.Parameter(torch.tensor(np.zeros((L), dtype='float32')))
W_1 = torch.randn(d_out, 3*d, requires_grad=True)
W_2 = torch.randn(L, d_out, requires_grad=True)

In [657]:
f = nn.ReLU()
cs = []
y_pred = []
m = nn.Softmax(dim=0)
for e in train_embedding:
    c = []
    p = []
    for t in range(1, len(e)-1):
        concat = torch.cat((e[t-1], e[t], e[t+1]), 0)
        p.append(f(torch.matmul(W_1, concat) + b_1))
    for i in range(d_out):
        c.append(max([s[i].item() for s in p]))
    c = torch.tensor(c).float()
    y = m(torch.matmul(W_2, c) + b_2)
    y_pred.append(y.tolist())

In [673]:
l_train = -torch.mean(t_train*torch_log(y_pred_train) + (1-t_train)*torch_log(1-y_pred_train))

In [676]:
y

tensor([1.2746e-37, 0.0000e+00, 3.2418e-29, 1.0000e+00],
       grad_fn=<SoftmaxBackward>)

In [675]:
l_train.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

In [715]:
epochs = 10
optimizer = optim.Adam([b_1, b_2, W_1, W_2], lr=0.001)
f = nn.ReLU()
y_pred_train = []
y_pred_test = []

m = nn.Softmax(dim=0)
for epoch in range(epochs):
    l_train = 0
    acc_train = 0
    for i in range(len(train_embedding)):
        e = train_embedding[i]
        c = []
        p = []
        for t in range(1, len(e)-1):
            concat = torch.cat((e[t-1], e[t], e[t+1]), 0)
            p.append(f(torch.matmul(W_1, concat) + b_1))
        for l in range(d_out):
            c.append(max([s[l].item() for s in p]))
        c = torch.tensor(c).float()
        y = m(torch.matmul(W_2, c) + b_2)
        l_train += -(t_train[i]*torch_log(y) + (1-t_train[i])*torch_log(1-y))
        if t_train[i].argmax(axis=0).item() == y.argmax(axis=0).item():
            acc_train += 1
    l_train = torch.mean(l_train/len(train_embedding))
    optimizer.zero_grad()
    l_train.backward()
    optimizer.step()

    accuracy_train = acc_train/len(train_embedding)

    l_test = 0
    acc_test = 0
    
    for i in range(len(test_embedding)):
        e = test_embedding[i]
        c = []
        p = []
        for t in range(1, len(e)-1):
            concat = torch.cat((e[t-1], e[t], e[t+1]), 0)
            p.append(f(torch.matmul(W_1, concat) + b_1))
        for l in range(d_out):
            c.append(max([s[l].item() for s in p]))
        c = torch.tensor(c).float()
        y = m(torch.matmul(W_2, c) + b_2)
        l_test += -(t_test[i]*torch_log(y) + (1-t_test[i])*torch_log(1-y))
        if t_test[i].argmax(axis=0).item() == y.argmax(axis=0).item():
            acc_train +=1
    l_test = torch.mean(l_test/len(test_embedding))
    acc_test = acc_test/len(test_embedding)

    print('loss_train:{:.3f}, acc_train:{:.3f}, loss_test:{:.3f}, acc_test:{:.3f}'.format(l_train.item(), accuracy_train ,l_test.item(), acc_test))

loss_train:10.524, acc_train:0.068, loss_test:10.441, acc_test:0.000
loss_train:10.512, acc_train:0.067, loss_test:10.432, acc_test:0.000


KeyboardInterrupt: 

In [61]:
import json

config_file = "/Users/ryomisawa/Library/Mobile Documents/com~apple~CloudDocs/pytorch/pytorch_advanced-master 2/8_nlp_sentiment_bert/weights/bert_config.json"

json_file = open(config_file, 'r')
config = json.load(json_file)

from attrdict import AttrDict

config = AttrDict(config)
config.hidden_size

768

In [72]:
class BertLayerNorm(nn.Module):

    def __init__(self, hidden_size, eps=1e-12):
        super(BertLayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(hidden_size))
        self.beta = nn.Parameter(torch.zeros(hidden_size))
        self.variance_epsilon = eps
    
    def forward(self, x):
        u = x.mean(-1, keepdim=True)
        s = (x - u).pow(2).mean(-1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.variance_epsilon)
        return self.gamma * x + self.beta

In [81]:
class BertEmbeddings(nn.Module):

    def __init__(self, config):
        super(BertEmbeddings, self).__init__()

        self.word_embeddings = nn.Embedding(
            config.vocab_size, config.hidden_size, padding_idx=0)
        
        self.position_embeddings = nn.Embedding(
            config.max_position_embeddings,config.hidden_size)
        
        self.LayerNorm = BertLayerNorm(config.hidden_size, eps=1e-12)

        self.dropout = nn.Dropout(config.hidden_dropout_prob)
    
    def forward(self, input_ids):
        words_embeddings = self.word_embeddings(input_ids)
        seq_length = input_ids.size(1)
        position_ids = torch.arange(
            seq_length, dtype=torch.long, device=input_ids.device)
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
        position_embeddings = self.position_embeddings(position_ids)

        embeddings = words_embeddings + position_embeddings

        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)

        return embeddings

In [77]:
import math 

In [87]:
class BertLayer(nn.Module):

    def __init__(self, config):
        super(BertLayer, self).__init__()

        self.attention = BertAttention(config)

        self.intermediate = BertIntermediate(config)

        self.output = BertOutput(config)
    
    def forward(self, hidden_states, attention_mask, attention_show_flg=False):
        
        if attention_show_flg == True:
            attention_output, attention_probs = self.attention(
                hidden_states, attention_mask, attention_show_flg)
            intermediate_output = self.intermediate(attention_output)
            layer_output = self.output(intermediate_output, attention_output)
            return layer_output, attention_probs
        
        elif attention_show_flg == False:
            attention_output = self.attention(
                hidden_states, attention_mask, attention_show_flg)
            intermediate_output = self.intermediate(attention_output)
            layer_output = self.output(intermediate_output, attention_output)
            
            return layer_output

class BertAttention(nn.Module):

    def __init__(self, config):
        super(BertAttention, self).__init__()
        self.selfattn = BertSelfAttention(config)
        self.output = BertSelfOutput(config)
    
    def forward(self, input_tensor, attention_mask, attention_show_flg=False):
        
        if attention_show_flg ==True:
            self_output, attention_probs = self.selfattn(input_tensor, 
                                                         attention_mask,
                                                         attention_show_flg)
            attention_output = self.output(self_output, input_tensor)
            return attention_output, attention_probs
        
        elif attention_show_flg == False:
            self_output = self.selfattn(input_tensor, attention_mask,
                                        attention_show_flg)
            attention_output = self.output(self_output, input_tensor)
            return attention_output 

class BertSelfAttention(nn.Module):

    def __init__(self, config):
        super(BertSelfAttention, self).__init__()

        self.num_attention_heads = config.num_attention_heads

        self.attention_head_size = int(
            config.hidden_size / config.num_attention_heads)
        
        self.all_head_size = self.num_attention_heads *  \
          self.attention_head_size
        
        self.query = nn.Linear(config.hidden_size, self.all_head_size)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)
        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)
    
    def transpose_for_scores(self, x):

        new_x_shape = x.size()[
            :-1] + (self.num_attention_heads, self.attention_head_size)
        x = x.view(*new_x_shape)
        return x.permute(0, 2, 1, 3)

    def forward(self, hidden_states, attention_mask, attention_show_flg=False):
        mixed_query_layer = self.query(hidden_states)
        mixed_key_layer = self.key(hidden_states)
        mixed_value_layer = self.value(hidden_states)

        query_layer = self.transpose_for_scores(mixed_query_layer)
        key_layer = self.transpose_for_scores(mixed_key_layer)
        value_layer = self.transpose_for_scores(mixed_value_layer)

        attention_scores = torch.matmul(
            query_layer, key_layer.transpose(-1, -2))
        attention_scores = attention_scores / \
            math.sqrt(self.attention_head_size)

        attention_scores = attention_scores + attention_mask

        attention_probs = nn.Softmax(dim=-1)(attention_scores)

        attention_probs = self.dropout(attention_probs)

        context_layer = torch.matmul(attention_probs, value_layer)

        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()
        new_context_layer_shape = context_layer.size()[
            :-2] + (self.all_head_size,)
        context_layer = context_layer.view(*new_context_layer_shape)

        if attention_show_flg == True:
            return context_layer, attention_probs
        
        elif attention_show_flg == False:
            return context_layer

class BertSelfOutput(nn.Module):

    def __init__(self, config):
        super(BertSelfOutput, self).__init__()

        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.LayerNorm = BertLayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states

def gelu(x):
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))

class BertIntermediate(nn.Module):

    def __init__(self, config):
        super(BertIntermediate, self).__init__()

        self.dense = nn.Linear(config.hidden_size, config.intermediate_size)

        self.intermediate_act_fn = gelu
    
    def forward(self, hidden_states):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.intermediate_act_fn(hidden_states)
        return hidden_states
    
class BertOutput(nn.Module):

    def __init__(self, config):
        super(BertOutput, self).__init__()

        self.dense = nn.Linear(config.intermediate_size, config.hidden_size)

        self.LayerNorm = BertLayerNorm(config.hidden_size, eps=1e-12)

        self.dropout = nn.Dropout(config.hidden_dropout_prob)
    
    def forward(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        return hidden_states

In [88]:
class BertEncoder(nn.Module):
    def __init__(self, config):
        super(BertEncoder, self).__init__()

        self.layer = nn.ModuleList([BertLayer(config)
                                    for _ in range(config.num_hidden_layers)])
    
    def forward(self, hidden_states, attention_mask,
                output_all_encoded_layers=True, attention_show_flg=False):
               
                all_encoder_layers = []

                for layer_module in self.layer:

                    if attention_show_flg == True:
                        hidden_states, attention_probs = layer_module(
                            hidden_states, attention_mask, attention_show_flg)

                    elif attention_show_flg == False:
                        hidden_states = layer_module(
                            hidden_states, attention_mask, attention_show_flg)
                    
                    if output_all_encoded_layers:
                        all_encoder_layers.append(hidden_states)
                
                if not output_all_encoded_layers:
                    all_encoder_layers.append(hidden_states)
                
                if attention_show_flg == True:
                    return all_encoder_layers, attention_probs
                elif attention_show_flg == False:
                    return all_encoder_layers

In [89]:
class BertPooler(nn.Module):
    def __init__(self, config):
        super(BertPooler, self).__init__()

        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.activation = nn.Tanh()
    
    def forward(self, hidden_states):
        first_token_tensor = hidden_states[:, 0]

        pooled_output = self.dense(first_token_tensor)

        pooled_output = self.activation(pooled_output)

        return pooled_output

In [90]:
input_ids = torch.LongTensor([[31, 51, 12, 23, 99], [15, 5, 1, 0, 0]])
print("入力の単語ID列のテンソルサイズ：", input_ids.shape)

# マスク
attention_mask = torch.LongTensor([[1, 1, 1, 1, 1], [1, 1, 1, 0, 0]])
print("入力のマスクのテンソルサイズ：", attention_mask.shape)

# 文章のID。2つのミニバッチそれぞれについて、0が1文目、1が2文目を示す
token_type_ids = torch.LongTensor([[0, 0, 1, 1, 1], [0, 1, 1, 1, 1]])
print("入力の文章IDのテンソルサイズ：", token_type_ids.shape)


# BERTの各モジュールを用意
embeddings = BertEmbeddings(config)
encoder = BertEncoder(config)
pooler = BertPooler(config)

# マスクの変形　[batch_size, 1, 1, seq_length]にする
# Attentionをかけない部分はマイナス無限にしたいので、代わりに-10000をかけ算しています
extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
extended_attention_mask = extended_attention_mask.to(dtype=torch.float32)
extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0
print("拡張したマスクのテンソルサイズ：", extended_attention_mask.shape)

# 順伝搬する
out1 = embeddings(input_ids)
print("BertEmbeddingsの出力テンソルサイズ：", out1.shape)

out2 = encoder(out1, extended_attention_mask)
# out2は、[minibatch, seq_length, embedding_dim]が12個のリスト
print("BertEncoderの最終層の出力テンソルサイズ：", out2[0].shape)

out3 = pooler(out2[-1])  # out2は12層の特徴量のリストになっているので一番最後を使用
print("BertPoolerの出力テンソルサイズ：", out3.shape)

入力の単語ID列のテンソルサイズ： torch.Size([2, 5])
入力のマスクのテンソルサイズ： torch.Size([2, 5])
入力の文章IDのテンソルサイズ： torch.Size([2, 5])
拡張したマスクのテンソルサイズ： torch.Size([2, 1, 1, 5])
BertEmbeddingsの出力テンソルサイズ： torch.Size([2, 5, 768])
BertEncoderの最終層の出力テンソルサイズ： torch.Size([2, 5, 768])
BertPoolerの出力テンソルサイズ： torch.Size([2, 768])


In [111]:
class BertModel(nn.Module):

    def __init__(self, config):
        super(BertModel, self).__init__()

        self.embeddings = BertEmbedding(config)
        self.encoder = BertEncoder(config)
        self.pooler = BertPooler(config)
    
    def forward(self, input_ids, attention_mask=None,output_all_encoded_layers=True, attention_show_flg=False):
                if attention_mask is None:
                    attention_mask = torch.ones_like(input_ids)
                
                extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)

                extended_attention_mask = extended_attention_mask.to(
                    dtype=torch.float32)
                extended_attention_mask = (1.0 - extended_attention_mask) * - 10000.0

                embedding_output = self.embeddings(input_ids)

                if attention_show_flg == True:
                    encoded_layers, attention_probs = self.encoder(embedding_output,
                                                                   extended_attention_mask,
                                                                   output_all_encoded_layers,
                                                                   attention_show_flg)
                elif attention_show_flg == False:
                    encoded_layers = self.encoder(embedding_output,
                                                  extended_attention_mask,
                                                  output_all_encoded_layers,
                                                  attention_show_flg)
                
                pooled_output = self.pooler(encoded_layers[-1])

                if not output_all_encoded_layers:
                    encoded_layers = encoded_layers[-1]
                
                if attention_show_flg == True:
                    return encoded_layers, pooled_output, attention_probs
                elif attention_show_flg == False:
                    return encoded_layers, pooled_output

In [112]:
input_ids = torch.LongTensor([[31, 51, 12, 23, 99], [15, 5, 1, 0, 0]])
attention_mask = torch.LongTensor([[1, 1, 1, 1, 1], [1, 1, 1, 0, 0]])
token_type_ids = torch.LongTensor([[0, 0, 1, 1, 1], [0, 1, 1, 1, 1]])

# BERTモデルを作る
net = BertModel(config)

# 順伝搬させる
encoded_layers, pooled_output, attention_probs= net(input_ids, attention_mask, output_all_encoded_layers=False, attention_show_flg=True)

print("encoded_layersのテンソルサイズ：", encoded_layers.shape)
print("pooled_outputのテンソルサイズ：", pooled_output.shape)
print("attention_probsのテンソルサイズ：", attention_probs.shape)

encoded_layersのテンソルサイズ： torch.Size([2, 5, 768])
pooled_outputのテンソルサイズ： torch.Size([2, 768])
attention_probsのテンソルサイズ： torch.Size([2, 12, 5, 5])


In [92]:
import collections

def load_vocab(vocab_file):
    vocab = collections.OrderedDict()
    ids_to_tokens = collections.OrderedDict()
    index = 0

    with open(vocab_file, 'r', encoding='utf-8') as reader:
        while True:
            token = reader.readline()
            if not token:
                break
            token = token.strip()

            vocab[token] = index
            ids_to_tokens[index] = token
            index += 1
    return vocab, ids_to_tokens

vocab_file = '/Users/ryomisawa/Library/Mobile Documents/com~apple~CloudDocs/pytorch/pytorch_advanced-master 2/8_nlp_sentiment_bert/vocab/bert-base-uncased-vocab.txt'
vocab, ids_to_tokens = load_vocab(vocab_file) 

In [93]:
import os
os.chdir('/Users/ryomisawa/Library/Mobile Documents/com~apple~CloudDocs/pytorch/pytorch_advanced-master 2/8_nlp_sentiment_bert')

from utils.tokenizer import BasicTokenizer, WordpieceTokenizer

class BertTokenizer(object):

    def __init__(self, vocab_file, do_lower_case=True):
        self.vocab, self.ids_to_tokens = load_vocab(vocab_file)

        never_split = ('[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]')

        self.basic_tokenizer = BasicTokenizer(do_lower_cae=do_lower_case,
                                              never_split=never_split)
        self.wordpiece_tokenizer = WordpieceTokenizer(vocab=self.vocab)
    
    def tokenize(self, text):
        split_tokens = []
        for token in self.basic_tokenizer.tokenize(text):
            for sub_token in self.wordpiece_tokenizer.tokenize(token):
                split_tokens.append(sub_token)
        return split_tokens
    
    def convert_tokens_to_ids(self, tokens):
        ids = []
        for token in tokens:
            ids.append(self.vocab[token])
        
        return ids
    
    def convert_ids_to_tokens(self, ids):
        tokens = []
        for i in ids:
            tokens.append(self.ids_to_tokens[i])
        return tokens

In [94]:
import re
import string

def preprocessing_text(text):
    text = re.sub('<br />', '', text)

    for p in string.punctuation:
        if (p == '.') or (p == ','):
            continue
        else:
            text = text.replace(p, ' ')
    
    text = text.replace('.', ' . ')
    text = text.replace(',', ' , ')
    return text

tokenizer_bert = BertTokenizer(
    vocab_file='/Users/ryomisawa/Library/Mobile Documents/com~apple~CloudDocs/pytorch/pytorch_advanced-master 2/8_nlp_sentiment_bert/vocab/bert-base-uncased-vocab.txt', do_lower_case=True)

def tokenizer_with_preprocessing(text, tokenizer=tokenizer_bert.tokenize):
    text = preprocessing_text(text)
    ret = tokenizer(text)
    return ret

In [113]:
from utils.bert import get_config, set_learned_params

config = get_config(file_path='/Users/ryomisawa/Library/Mobile Documents/com~apple~CloudDocs/pytorch/pytorch_advanced-master 2/8_nlp_sentiment_bert/weights/bert_config.json')

net_bert = BertModel(config)

net_bert = set_learned_params(
    net_bert, weights_path = '/Users/ryomisawa/Library/Mobile Documents/com~apple~CloudDocs/pytorch/pytorch_advanced-master 2/8_nlp_sentiment_bert/weights/pytorch_model.bin')

bert.embeddings.word_embeddings.weight→embeddings.word_embeddings.weight
bert.embeddings.position_embeddings.weight→embeddings.position_embeddings.weight
bert.embeddings.token_type_embeddings.weight→embeddings.LayerNorm.gamma
bert.embeddings.LayerNorm.gamma→embeddings.LayerNorm.beta
bert.embeddings.LayerNorm.beta→encoder.layer.0.attention.selfattn.query.weight
bert.encoder.layer.0.attention.self.query.weight→encoder.layer.0.attention.selfattn.query.bias
bert.encoder.layer.0.attention.self.query.bias→encoder.layer.0.attention.selfattn.key.weight
bert.encoder.layer.0.attention.self.key.weight→encoder.layer.0.attention.selfattn.key.bias
bert.encoder.layer.0.attention.self.key.bias→encoder.layer.0.attention.selfattn.value.weight
bert.encoder.layer.0.attention.self.value.weight→encoder.layer.0.attention.selfattn.value.bias
bert.encoder.layer.0.attention.self.value.bias→encoder.layer.0.attention.output.dense.weight
bert.encoder.layer.0.attention.output.dense.weight→encoder.layer.0.attention.

RuntimeError: Error(s) in loading state_dict for BertModel:
	size mismatch for embeddings.LayerNorm.gamma: copying a param with shape torch.Size([2, 768]) from checkpoint, the shape in current model is torch.Size([768]).
	size mismatch for encoder.layer.0.attention.selfattn.query.weight: copying a param with shape torch.Size([768]) from checkpoint, the shape in current model is torch.Size([768, 768]).
	size mismatch for encoder.layer.0.attention.selfattn.query.bias: copying a param with shape torch.Size([768, 768]) from checkpoint, the shape in current model is torch.Size([768]).
	size mismatch for encoder.layer.0.attention.selfattn.key.weight: copying a param with shape torch.Size([768]) from checkpoint, the shape in current model is torch.Size([768, 768]).
	size mismatch for encoder.layer.0.attention.selfattn.key.bias: copying a param with shape torch.Size([768, 768]) from checkpoint, the shape in current model is torch.Size([768]).
	size mismatch for encoder.layer.0.attention.selfat

In [14]:
class BertForNLP(nn.Module):

    def __init__(self, net_bert):
        super(BertForNLP, self).__init__()

        self.bert = net_bert

        self.cls = nn.Linear(in_features=768, out_features=4)

        nn.init.normal_(self.cls.weight, std=0.02)
        nn.init.normal_(self.cls.bias, 0)
    
    def forward(self, input_ids, token_type_ids=None,
                attention_mask=None, output_all_encoded_layers=False,
                attention_show_flg=False):

                if attention_show_flg == True:
                    encoded_layers, pooled_output, attention_probs = self.bert(
                        input_ids, token_type_ids, attention_mask,
                        output_all_encoded_layers, attention_show_flg)
                
                elif attention_show_flg == False:
                     encoded_layers, pooled_output, attention_probs = self.bert(
                        input_ids, token_type_ids, attention_mask,
                        output_all_encoded_layers, attention_show_flg)
                
                vec_0 = encoded_layers[:, 0, :]
                vec_0 = vec_0.view(-1, 768)
                out = self.cls(vec_0)

                if attention_show_flg == True:
                    return out, attention_probs
                
                if attention_show_flg == False:
                    return out

In [15]:
net = BertForNLP(net_bert)

net.train()

BertForNLP(
  (bert): BertModel(
    (embedding): BertEmbedding(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (selfattn): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
          )
          (intermediate): BertIntermediate(

In [16]:
for name, param in net.named_parameters():
    param.requires_grad = False

for name, param in net.bert.encoder.layer[-1].named_parameters():
    param.requires_grad = True

for name, param in net.cls.named_parameters():
    param.requires_grad = True

In [ ]:
optimizer = optim.Adam([
    {'params': net.bert.encoder.layer[-1].parameters(), 'lr':5e-5},
    {'params': net.cls.parameters(), 'lr':5e-5}
], betas=(0.9, 0.999))

criterion = nn.CrossEntropyLoss()

In [124]:
col_names = ['ID', 'TITLE', 'URL', 'PUBLISHER', 'CATEGORY', 'STORY', 'HOSTNAME', 'TIMESTAMP']
df_news = pd.read_csv('/Users/ryomisawa/Downloads/NewsAggregatorDataset/newsCorpora.csv', sep = '\t', names=col_names)

df_news = df_news[(df_news['PUBLISHER'] == 'Reuters') | (df_news['PUBLISHER'] ==  'Huffington Post')|(df_news['PUBLISHER'] == 'Businessweek') | (df_news['PUBLISHER'] == 'Contactmusic.com') |(df_news['PUBLISHER'] == 'Daily Mail')].sample(frac=1, random_state=0).reset_index()

df_news['CATEGORY'].replace('b', 0, inplace=True)
df_news['CATEGORY'].replace('t', 1, inplace=True)
df_news['CATEGORY'].replace('e', 2, inplace=True)
df_news['CATEGORY'].replace('m', 3, inplace=True)

In [48]:
max_length = 256

TEXT = torchtext.data.Field(sequential=True,
                            tokenize=tokenizer_with_preprocessing, use_vocab=True,
                            lower=True, include_lengths=True, batch_first=True,
                            fix_length=max_length, init_token="[CLS]",
                            eos_token="[SEP]", pad_token='[PAD]',
                            unk_token='[UNK]')
LABEL = torchtext.data.Field(sequential=False, use_vocab=False)

In [52]:
import random
train_val_ds, test_ds = torchtext.data.TabularDataset.splits(
    path='/Users/ryomisawa/nlp_tutorial/', train='news_train.csv',
    test='news_test.csv',format='csv', 
    fields=[('Text',TEXT), ('Label', LABEL)])

train_ds, val_ds = train_val_ds.split(
    split_ratio=0.89, random_state=random.seed(1234))

In [56]:
vocab_bert, ids_to_tokens_bert = load_vocab(
    vocab_file='/Users/ryomisawa/Library/Mobile Documents/com~apple~CloudDocs/pytorch/pytorch_advanced-master 2/8_nlp_sentiment_bert/vocab/bert-base-uncased-vocab.txt')

TEXT.build_vocab(train_ds, min_freq=1)
TEXT.vocab.stoi = vocab_bert

In [59]:
batch_size = 32

train_dl = torchtext.data.Iterator(
    train_ds, batch_size=batch_size, train=True)

val_dl = torchtext.data.Iterator(
    val_ds, batch_size=batch_size, train=False, sort=False)

test_dl = torchtext.data.Iterator(
    test_ds, batch_size=batch_size, train=False, sort=False)

dataloaders_dict = {'train': train_dl, 'val': val_dl}

In [119]:
batch = next(iter(train_dl))
print(batch.Text)
print(batch.Label)

ValueError: invalid literal for int() with base 10: 'FOREX-Dollar stung by Fed minutes, Aussie rises on solid jobs data'